In [18]:
# Import required libraries with updated imports
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch


# Define model ID and cache dir
model_id = "meta-llama/Llama-3.2-1B-Instruct"
cache_dir = "./hf_models"

# Check device availability -this is for my macbook / you can use cuda if on windows
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    cache_dir=cache_dir
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device in ["mps", "cuda"] else torch.float32,
    device_map="auto",
    cache_dir=cache_dir
)
# You can change this
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1,
)

 
llm = HuggingFacePipeline(
    pipeline=text_generation_pipeline,
    model_kwargs={"return_full_text": False}
)


Using device: mps


In [ ]:
targeted_resume_prompt = ChatPromptTemplate.from_template("""
You are a professional resume specialist helping create tailored experience sections for technical professionals.

[INSTRUCTION]
Identify the distinct roles/positions from the input text based on:
1. Organization names
2. Job titles
3. Time periods
4. Technical skills                                                          
                                     
Then, for EACH identified position, create an experience section following this EXACT format:

[Job Title]: [Specific Role]
- **Company/Organization**: [Organization Name]
- **Duration**: [Start Date - End Date]
- **Responsibilities and Achievements**:
  - Point 1
  - Point 2
  - Point 3

[END INSTRUCTION]
Do not include the instruction in the response.

Input:
\"\"\"{context}\"\"\"
""")

input_text = """
Justin has been an excellent python programmer at Facebook from 2015-2017. He has done excepetional work with writing code and hold team meetings.
"""

# Create output parser
output_parser = StrOutputParser()

# Create the chain correctly (only once)
resume_chain = targeted_resume_prompt | llm | output_parser

# Process the input with debug information
print("Invoking the chain...")
try:
    response = resume_chain.invoke({"context": input_text})
    print("Chain invoked successfully")
    
    print(response)
    
except Exception as e:
    print(f"Error invoking chain: {e}")

Invoking the chain...
Chain invoked successfully
Human: 
You are a professional resume specialist helping create tailored experience sections for technical professionals.

[INSTRUCTION]
Identify the distinct roles/positions from the input text based on:
1. Organization names
2. Job titles
3. Time periods
4. Technical skills                                                          
                                     
Then, for EACH identified position, create an experience section following this EXACT format:

[Job Title]: [Specific Role]
- **Company/Organization**: [Organization Name]
- **Duration**: [Start Date - End Date]
- **Responsibilities and Achievements**:
  - Point 1
  - Point 2
  - Point 3

[END INSTRUCTION]
Do not include the instruction in the response.

Input:
"""
Justin has been an excellent python programmer at Facebook from 2015-2017. He has done excepetional work with writing code and hold team meetings.
"""
"""

## Justin's Experience
### Python Programmer
#### Face